In [9]:
# Add helper functions
import json

FILE_PATH = "/content/drive/MyDrive/Projects/Coffee Machine/oop_version/data.json"

def load_data(filename_or_path=FILE_PATH):
  with open(filename_or_path, "r") as f:
    return json.load(f)

def save_data(data, filename_or_path=FILE_PATH):
  with open(filename_or_path, "w") as f:
    json.dump(data, f, indent=4)

# Define "MenuItem" class
class MenuItem:
  def __init__(self, name, water, milk, coffee, cost):
    self.name = name
    self.cost = cost
    self.ingredients = {
        "water": water,
        "milk": milk,
        "coffee": coffee
    }

# Define "Menu" class
class Menu:
  def __init__(self):
    self.menu = [
        MenuItem(name="latte", water=200, milk=150, coffee=24, cost=2.5),
        MenuItem(name="espresso", water=50, milk=0, coffee=18, cost=1.5),
        MenuItem(name="cappucino", water=250, milk=50, coffee=24, cost=3)
    ]

  def get_items(self):
    options = ""
    for item in self.menu:
      options += f"{item.name}/"
    return options

  def find_drink(self, order_name):
      for item in self.menu:
        if item.name == order_name:
          return item
      print("Sorry that item is not available.")

# Define "CoffeeMaker" class
class CoffeeMaker:
  def __init__(self, data):
    self.resources = data["resources"]

  def report(self):
    print(f"Water: {self.resources['water']}ml")
    print(f"Milk: {self.resources['milk']}ml")
    print(f"Coffee: {self.resources['coffee']}g")

  def is_resource_sufficient(self, drink):
    can_make = True
    for item in drink.ingredients:
      if drink.ingredients[item] > self.resources[item]:
        print(f"Sorry there is not enough {item}.")
        can_make = False
    return can_make

  def make_coffee(self, order):
    for item in order.ingredients:
      self.resources[item] -= order.ingredients[item]
    print(f"Here is your {order.name} ☕️. Enjoy!")

# Define "MoneyMachine" class
class MoneyMachine:
  CURRENCY = "$"
  COIN_VALUES = {
      "quarters": 0.25,
      "dimes": 0.10,
      "nickles": 0.05,
      "pennies": 0.01
  }

  def __init__(self, data):
    self.profit = data["sales"]["profit"]
    self.money_received = 0
    self.data = data

  def report(self):
    print(f"Money: {self.CURRENCY}{self.profit}")

  def process_coins(self):
    print("Please insert coins.")
    for coin in self.COIN_VALUES:
      self.money_received += int(input(f"How many {coin}?: ")) * self.COIN_VALUES[coin]
    return self.money_received

  def make_payment(self, cost):
    self.process_coins()
    if self.money_received >= cost:
      change = round(self.money_received - cost, 2)
      print(f"Here is {self.CURRENCY}{change} in change.")
      self.profit += cost
      self.data["sales"]["profit"] = self.profit
      self.money_received = 0
      return True
    else:
      print("Sorry that's not enough money. Money refunded.")
      self.money_received = 0
      return False

data = load_data(FILE_PATH)

money_machine = MoneyMachine(data)
coffee_maker = CoffeeMaker(data)
menu = Menu()

is_on = True

while is_on:
  options = menu.get_items()
  choice = input(f"What would you like {options}: ")
  if choice == 'off':
    save_data(data)
    is_on = False
  elif choice == 'report':
    coffee_maker.report()
    money_machine.report()
  else:
    drink = menu.find_drink(choice)
    if drink:
      if coffee_maker.is_resource_sufficient(drink) and money_machine.make_payment(drink.cost):
        coffee_maker.make_coffee(drink)
        save_data(data)

What would you like latte/espresso/cappucino/: latte
Please insert coins.
How many quarters?: 100
How many dimes?: 250
How many nickles?: 23
How many pennies?: 5
Here is $48.7 in change.
Here is your latte ☕️. Enjoy!
What would you like latte/espresso/cappucino/: report
Water: 600ml
Milk: 700ml
Coffee: 152g
Money: $5.0
What would you like latte/espresso/cappucino/: 0
Sorry that item is not available.
What would you like latte/espresso/cappucino/: off
